In [1]:
import getpass 
password = getpass.getpass('Enter your password:')
import pandas as pd
data=pd.read_csv("D:\py\Historical daily data.csv",parse_dates=['SDATE'])

Enter your password:········


C:\Users\Shivam Rathour\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (12,13,14,15,17,18,19,23,28,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Send alert for Null value of anyone SDATE,REPL_PN,GA,GROUP

In [2]:
data["Null Column"]=''
data.loc[data["SDATE"].isnull()==True, "Null Column"] =data["Null Column"]+"SDATE, "
data.loc[data["REPL_PN"].isnull()==True, "Null Column"] =data["Null Column"]+"REPL_PN, "
data.loc[data["GA_GROUP"].isnull()==True, "Null Column"] =data["Null Column"]+"GA_GROUP, "


In [3]:
null_records=data[(data["REPL_PN"].isnull()==True)|(data["GA_GROUP"].isnull()==True)|(data["SDATE"].isnull()==True)]
null_records.to_excel("nullrecords.xlsx")

In [4]:
import pandas as pd
import getpass 
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
filename="nullrecords.xlsx"
df=pd.read_excel(filename)
row=list(df.shape)
k=row[0]
def send_Email(k,File_Name):
  if(k > 0):
    port = 587
    smtp_server = "smtp.gmail.com"
    login = "shivam.rathour@gainops.com" # paste your login generated by Mailtrap
#     password = getpass.getpass('Enter your password:') # paste your password generated by Mailtrap
    subject = "Testing From GaiOps with Python"
    sender_email = "shivam.rathour@gainops.com"
    receiver_email = "shivamra8hour@gmail.com"
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    # Add body to email
    body = "These records contain null infomation of require columns and we put the default value for these records"
    message.attach(MIMEText(body, "plain"))
#     filename = "D:\py\productMaster.xlsx"
    # Open PDF file in binary mode
    # We assume that the file is in the directory where you run your Python script from
    with open(filename, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    # Encode to base64
    encoders.encode_base64(part)
    # Add header
    part.add_header("Content-Disposition", f"attachment; filename= {filename}")
    # Add attachment to your message and convert it to string
    message.attach(part)

    text = message.as_string()
    # send your email
    server=smtplib.SMTP('smtp.gmail.com',587)
    server.starttls()
    server.login(login,password)
    print('Login Success')
    server.sendmail(sender_email, receiver_email, text)
    print('email has been sent to',receiver_email)    
  else:
    print('Email Alert Not Sent')
send_Email(k,filename)    

Login Success
email has been sent to shivamra8hour@gmail.com


# Select require column

In [5]:
d_columns=['SDATE','GA_GROUP','REPL_PN',
       'SUPPLY_PLAN', 'SUPPLY_ARRIVAL', 'HYBRID_FCST', 'BOH_INTRANSIT',
       'TOTAL_SUPPLY', 'SUPPLY_GAP']

# Fill NaN with default value

In [6]:
data["SDATE"].fillna("9999-09-09",inplace=True)
data["GA_GROUP"].fillna('UNKNOWN',inplace=True)
data["REPL_PN"].fillna('UNKNOWN',inplace=True)


Check null Records

In [7]:
data[(data["GA_GROUP"].isnull()==True)]

,SDATE,OSC,PSI_GROUP,GA_GROUP,END_CUST_HEADER,REPL_PN,SUPPLY_PLAN,SUPPLY_ARRIVAL,HYBRID_FCST,BOH_INTRANSIT,...,DIE_STACK1_COUNT,DIE_STACK2,DIE_STACK2_COUNT,PWB,PACKAGE_SIZE,DESCRIPTION,NAND_GRADE,CONTROLLER_TYPE,W_CODE,Null Column


In [8]:
# Extract require Column
data=data[d_columns]
# Extract left 12 in REPL_PN
data['REPL_PN']=data['REPL_PN'].str[:12]
# Extract Month and year from SDATE
data['SDATE']=pd.to_datetime(data['SDATE']).dt.to_period('M')
print(data.shape)
data.head()

(223602, 9)


,SDATE,GA_GROUP,REPL_PN,SUPPLY_PLAN,SUPPLY_ARRIVAL,HYBRID_FCST,BOH_INTRANSIT,TOTAL_SUPPLY,SUPPLY_GAP
0,2020-02,Cisco,SDFBE06JAA01,90.0,NaN,NaN,0.0,0.0,0.0
1,2020-02,Microsoft,KBG40ZNS128G,NaN,NaN,330.0,0.0,0.0,-330.0
2,2020-02,Dell,SDFBB85DUB01,NaN,NaN,60.0,0.0,0.0,-60.0
3,2020-02,HPE,SDFBD05CAA01,NaN,NaN,4500.0,0.0,0.0,-4500.0
4,2020-02,Microsoft,KBG40ZNS256G,NaN,NaN,380.0,0.0,0.0,-380.0


# Product Analysis and Cleaning

In [9]:
product=pd.read_excel("D:\py\productMaster.xlsx")
product=product[['PART_NUMBER','W_CODE']]
product['PART_NUMBER']=product['PART_NUMBER'].str[:12]


In [10]:
# product[product['W_CODE'].isnull()]

In [11]:
# Drop Duplicate records from where key PART_NUMBER and W_CODE
product.drop_duplicates(['PART_NUMBER','W_CODE'],inplace=True)

In [12]:
# Find Duplicate records bassed on only joining key
product[product.duplicated('PART_NUMBER',keep=False)].shape

(148, 2)

In [13]:
Dup=product[product.duplicated('PART_NUMBER',keep=False)]
dup=list(Dup['PART_NUMBER'].unique())

In [14]:
# product 1 contain only that unqique records or that records not hold duplicate value
product1=product[~product.PART_NUMBER.isin(dup)]
product2=Dup[Dup['W_CODE']!='-']

In [15]:
product2.shape

(74, 2)

In [16]:
product3=pd.concat([product1, product2])
product3.shape

(5340, 2)

In [17]:
product3[product3.duplicated('PART_NUMBER',keep=False)]

,PART_NUMBER,W_CODE


In [18]:
# product[product['W_CODE']==product.PART_NUMBER.isin(['KHK6UVSE1T92','KHK6YVSE1T92'])]
# product3.loc[product3.PART_NUMBER.isin(['KHK6UVSE1T92','KHK6YVSE1T92']), 'W_CODE'] = "HK6_15"

In [19]:
# product3[product3.PART_NUMBER.isin(['KHK6UVSE1T92','KHK6YVSE1T92'])]

In [20]:
product3.drop_duplicates(['PART_NUMBER','W_CODE'],inplace=True)

In [21]:
product3[product3.duplicated('PART_NUMBER',keep=False)]

,PART_NUMBER,W_CODE


In [22]:
product3[product3['W_CODE'].isnull()]

,PART_NUMBER,W_CODE


Update value of specific cell bassed on another column 

In [23]:
product3.shape

(5340, 2)

In [24]:
Product_data=data.merge(product3,left_on='REPL_PN',right_on='PART_NUMBER', how='left')
Product_data1=data.merge(product3,left_on='REPL_PN',right_on='PART_NUMBER', how='left',indicator=True)
# Product_data.columns
notinproduct=Product_data1[Product_data1['_merge']=='left_only']
notinproduct.to_excel("notinproduct.xlsx")

In [25]:
import pandas as pd
import getpass 
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
filename="notinproduct.xlsx"
df=pd.read_excel(filename)
row=list(df.shape)
k=row[0]
def send_Email(k,File_Name):
  if(k > 0):
    port = 587
    smtp_server = "smtp.gmail.com"
    login = "shivam.rathour@gainops.com" # paste your login generated by Mailtrap
#     password = getpass.getpass('Enter your password:') # paste your password generated by Mailtrap
    subject = "Testing From GaiOps with Python"
    sender_email = "shivam.rathour@gainops.com"
    receiver_email = "shivamra8hour@gmail.com"
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    # Add body to email
    body = "This file contain those records exist in Historical Daily table but not in product table"
    message.attach(MIMEText(body, "plain"))
#     filename = "D:\py\productMaster.xlsx"
    # Open PDF file in binary mode
    # We assume that the file is in the directory where you run your Python script from
    with open(filename, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    # Encode to base64
    encoders.encode_base64(part)
    # Add header
    part.add_header("Content-Disposition", f"attachment; filename= {filename}")
    # Add attachment to your message and convert it to string
    message.attach(part)

    text = message.as_string()
    # send your email
    server=smtplib.SMTP('smtp.gmail.com',587)
    server.starttls()
    server.login(login,password)
    print('Login Success')
    server.sendmail(sender_email, receiver_email, text)
    print('email has been sent to',receiver_email)    
  else:
    print('Email Alert Not Sent')
send_Email(k,filename)    

Login Success
email has been sent to shivamra8hour@gmail.com


In [26]:
Product_data.loc[Product_data['W_CODE'].isnull(),'W_CODE']='UNKNOWN'
# Product_data[Product_data['W_CODE']=='UNKNOWN']


In [27]:
pd=Product_data[['SDATE', 'GA_GROUP','REPL_PN', 'W_CODE','SUPPLY_GAP']]
pd.shape

(223602, 5)

In [28]:
product_data_groupby=pd.groupby(['SDATE','W_CODE','GA_GROUP'],as_index=False).sum()
CM=product_data_groupby[product_data_groupby["SDATE"]>='2018-08']


In [29]:
# CM

In [30]:
Product_data.shape

(223602, 11)

In [31]:
Product_data.head()

,SDATE,GA_GROUP,REPL_PN,SUPPLY_PLAN,SUPPLY_ARRIVAL,HYBRID_FCST,BOH_INTRANSIT,TOTAL_SUPPLY,SUPPLY_GAP,PART_NUMBER,W_CODE
0,2020-02,Cisco,SDFBE06JAA01,90.0,NaN,NaN,0.0,0.0,0.0,SDFBE06JAA01,W0FD
1,2020-02,Microsoft,KBG40ZNS128G,NaN,NaN,330.0,0.0,0.0,-330.0,KBG40ZNS128G,K
2,2020-02,Dell,SDFBB85DUB01,NaN,NaN,60.0,0.0,0.0,-60.0,SDFBB85DUB01,W0FA
3,2020-02,HPE,SDFBD05CAA01,NaN,NaN,4500.0,0.0,0.0,-4500.0,SDFBD05CAA01,W0F8
4,2020-02,Microsoft,KBG40ZNS256G,NaN,NaN,380.0,0.0,0.0,-380.0,KBG40ZNS256G,M


In [32]:
# mask1=Product_data.W_CODE.isin(['NULL','UNKNOWN'])
# Product_data[mask1]
# Product_data.loc[Product_data.W_CODE.isin(['NULL','-']),'W_CODE']="UNKNOWN"
Product_data=Product_data[Product_data["SDATE"]>='2018-08']
# Product_data.to_excel("product_Data.xlsx")


In [33]:
Product_data.shape

(189022, 11)

In [34]:
cm=Product_data.groupby(['SDATE','W_CODE','GA_GROUP'],as_index=False).sum()
# cm=product_data_groupby[product_data_groupby["SDATE"]>='2018-08']

In [35]:
cm.columns

Index(['SDATE', 'W_CODE', 'GA_GROUP', 'SUPPLY_PLAN', 'SUPPLY_ARRIVAL',
       'HYBRID_FCST', 'BOH_INTRANSIT', 'TOTAL_SUPPLY', 'SUPPLY_GAP'],
      dtype='object')

In [36]:
cm_1=cm
cm['FDATE']=cm['SDATE']+1


In [37]:
# cm_1.drop('SDATE1',axis=1,inplace=True)
CM_1=cm.merge(cm_1,left_on=['FDATE','W_CODE','GA_GROUP'],right_on=['SDATE','W_CODE','GA_GROUP'], how='left')

In [38]:
CM_1.columns

Index(['SDATE_x', 'W_CODE', 'GA_GROUP', 'SUPPLY_PLAN_x', 'SUPPLY_ARRIVAL_x',
       'HYBRID_FCST_x', 'BOH_INTRANSIT_x', 'TOTAL_SUPPLY_x', 'SUPPLY_GAP_x',
       'FDATE_x', 'SDATE_y', 'SUPPLY_PLAN_y', 'SUPPLY_ARRIVAL_y',
       'HYBRID_FCST_y', 'BOH_INTRANSIT_y', 'TOTAL_SUPPLY_y', 'SUPPLY_GAP_y',
       'FDATE_y'],
      dtype='object')

In [39]:
CM_1.head(1)

,SDATE_x,W_CODE,GA_GROUP,SUPPLY_PLAN_x,SUPPLY_ARRIVAL_x,HYBRID_FCST_x,BOH_INTRANSIT_x,TOTAL_SUPPLY_x,SUPPLY_GAP_x,FDATE_x,SDATE_y,SUPPLY_PLAN_y,SUPPLY_ARRIVAL_y,HYBRID_FCST_y,BOH_INTRANSIT_y,TOTAL_SUPPLY_y,SUPPLY_GAP_y,FDATE_y
0,2018-08,-,Avnet,0.0,0.0,40.0,0.0,0.0,-40.0,2018-09,2018-09,0.0,0.0,30.0,0.0,0.0,-30.0,2018-10


In [40]:
notincm=cm.merge(cm_1,left_on=['FDATE','W_CODE','GA_GROUP'],right_on=['SDATE','W_CODE','GA_GROUP'], how='right',indicator=True)

In [41]:
notincm=notincm[notincm['_merge']=='right_only']

In [42]:
notincm.shape

(2594, 19)

In [43]:
# notincm.to_excel (r'D:\py\NotInCm.xlsx', index =False, header=True)

In [44]:
# cm.to_excel (r'D:\py\groupBy.xlsx', index =False, header=True)

In [45]:
C_columns=['SDATE_x', 'W_CODE', 'GA_GROUP', 'SUPPLY_PLAN_x', 'SUPPLY_ARRIVAL_x',
       'HYBRID_FCST_x', 'BOH_INTRANSIT_x', 'TOTAL_SUPPLY_x', 'SUPPLY_GAP_x','SDATE_y', 'SUPPLY_PLAN_y', 'SUPPLY_ARRIVAL_y',
       'HYBRID_FCST_y', 'BOH_INTRANSIT_y', 'TOTAL_SUPPLY_y', 'SUPPLY_GAP_y']

In [46]:
notincm.columns
CM_1=CM_1[C_columns]

In [47]:
notincm['SDATE_x']=notincm['SDATE_y']-1

In [48]:
# notincm=notincm[notincm['SDATE_x']!= '2018-07']

In [49]:
notincm.head()

,SDATE_x,W_CODE,GA_GROUP,SUPPLY_PLAN_x,SUPPLY_ARRIVAL_x,HYBRID_FCST_x,BOH_INTRANSIT_x,TOTAL_SUPPLY_x,SUPPLY_GAP_x,FDATE_x,SDATE_y,SUPPLY_PLAN_y,SUPPLY_ARRIVAL_y,HYBRID_FCST_y,BOH_INTRANSIT_y,TOTAL_SUPPLY_y,SUPPLY_GAP_y,FDATE_y,_merge
5539,2018-07,-,Avnet,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2018-08,0.0,0.0,40.0,0.0,0.0,-40.0,2018-09,right_only
5540,2018-07,-,Cisco,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2018-08,4860.0,4860.0,3218.0,712.0,5572.0,2354.0,2018-09,right_only
5541,2018-07,-,Dell,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2018-08,0.0,0.0,273359.0,3645.0,3645.0,-269714.0,2018-09,right_only
5542,2018-07,-,EMC,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2018-08,0.0,0.0,6505.0,499.0,499.0,-6006.0,2018-09,right_only
5543,2018-07,-,Generic,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2018-08,228.0,463.0,0.0,50.0,513.0,513.0,2018-09,right_only


In [50]:
notincm=notincm[['SDATE_x', 'W_CODE', 'GA_GROUP', 'SUPPLY_PLAN_x', 'SUPPLY_ARRIVAL_x',
       'HYBRID_FCST_x', 'BOH_INTRANSIT_x', 'TOTAL_SUPPLY_x', 'SUPPLY_GAP_x',
        'SDATE_y', 'SUPPLY_PLAN_y', 'SUPPLY_ARRIVAL_y',
       'HYBRID_FCST_y', 'BOH_INTRANSIT_y', 'TOTAL_SUPPLY_y', 'SUPPLY_GAP_y']]

In [51]:
CM_1=CM_1[['SDATE_x', 'W_CODE', 'GA_GROUP', 'SUPPLY_PLAN_x', 'SUPPLY_ARRIVAL_x',
       'HYBRID_FCST_x', 'BOH_INTRANSIT_x', 'TOTAL_SUPPLY_x', 'SUPPLY_GAP_x',
        'SDATE_y', 'SUPPLY_PLAN_y', 'SUPPLY_ARRIVAL_y',
       'HYBRID_FCST_y', 'BOH_INTRANSIT_y', 'TOTAL_SUPPLY_y', 'SUPPLY_GAP_y']]

In [52]:
import pandas as pd
result=pd.concat([notincm, CM_1])

In [53]:
result.shape

(10727, 16)